In [1]:
#pip install kaggle
#pip install --upgrade boto3 botocore urllib3
import pandas as pd
!kaggle datasets download -d rmisra/news-category-dataset --force


Dataset URL: https://www.kaggle.com/datasets/rmisra/news-category-dataset
License(s): Attribution 4.0 International (CC BY 4.0)




  0%|          | 0.00/26.5M [00:00<?, ?B/s]
100%|##########| 26.5M/26.5M [00:00<00:00, 331MB/s]


In [2]:
from zipfile import ZipFile
with ZipFile(r'C:\Users\91904\news-category-dataset.zip','r')as zip_ref:
    zip_ref.extractall()
    zip_ref.close() 

In [3]:
import pandas as pd
df=pd.read_json(r'C:\Users\91904\News_Category_Dataset_v3.json',lines=True)
df.to_csv(r'C:\Users\91904\news-category.csv',index=False)
df.info()
df.describe()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB
                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            he

In [4]:
df.describe()

,date
count,209527
mean,2015-04-30 00:44:14.344308736
min,2012-01-28 00:00:00
25%,2013-08-10 00:00:00
50%,2015-03-16 00:00:00
75%,2016-11-01 00:00:00
max,2022-09-23 00:00:00


In [10]:
from botocore.config import Config
import boto3

config = Config(
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    },
    connect_timeout=30,
    read_timeout=120
)
acc_key="YOUR ACCESS KEY"
sec_key="YOUR SECRET KEY"
s3 = boto3.client('s3', config=config,aws_access_key_id=acc_key,aws_secret_access_key=sec_key)

file = r"C:\Users\91904\news-category.csv"
s3.upload_file(file, 'news-senticonomy-bucket', 'news-senti.csv')


In [11]:
import pandas as pd
df=pd.read_csv(r"C:\Users\91904\news-category.csv")
df=df[df['category'].isin(['SPORTS','EDUCATION','BUSINESS','POLITICS','TECH','SCIENCE','MONEY'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53751 entries, 13 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   link               53751 non-null  object
 1   headline           53749 non-null  object
 2   category           53751 non-null  object
 3   short_description  48550 non-null  object
 4   authors            47071 non-null  object
 5   date               53751 non-null  object
dtypes: object(6)
memory usage: 2.9+ MB


In [12]:
df.isna().sum()

link                    0
headline                2
category                0
short_description    5201
authors              6680
date                    0
dtype: int64

In [13]:
df=df.dropna(how='any',subset=['headline','category','date'])

In [14]:
print(df)

                                                     link  \
13      https://www.huffpost.com/entry/twitch-streamer...   
17      https://www.huffpost.com/entry/dodgers-basebal...   
21      https://www.huffpost.com/entry/biden-us-forces...   
24      https://www.huffpost.com/entry/ukraine-festiva...   
26      https://www.huffpost.com/entry/2022-wnba-final...   
...                                                   ...   
209522  https://www.huffingtonpost.com/entry/rim-ceo-t...   
209523  https://www.huffingtonpost.com/entry/maria-sha...   
209524  https://www.huffingtonpost.com/entry/super-bow...   
209525  https://www.huffingtonpost.com/entry/aldon-smi...   
209526  https://www.huffingtonpost.com/entry/dwight-ho...   

                                                 headline  category  \
13      Twitch Bans Gambling Sites After Streamer Scam...      TECH   
17      Maury Wills, Base-Stealing Shortstop For Dodge...    SPORTS   
21      Biden Says U.S. Forces Would Defend Taiwan If 

In [17]:
df['date']=pd.to_datetime(df['date'])

In [18]:
df.to_csv(r'c:\users\91904\after-news.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53749 entries, 13 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   link               53749 non-null  object        
 1   headline           53749 non-null  object        
 2   category           53749 non-null  object        
 3   short_description  48550 non-null  object        
 4   authors            47069 non-null  object        
 5   date               53749 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 2.9+ MB


In [26]:
import requests
from botocore.config import Config


# Your NewsAPI key
api_key = 'YOUR API KEY'

def get_news(query, from_date, to_date, api_key):
    url = f'https://newsapi.org/v2/everything?q={query}&from={from_date}&to={to_date}&sortBy=publishedAt&apiKey={api_key}'
    response = requests.get(url)
    data = response.json()
    all_data=[]


    if 'articles' in data:
               
        items=data['articles']
        for item in items:
          
           news_info={'link':item['url'],
                      'headline':item['title'],
                      'category':query,
                     'short_description':item['description'],
                    
                     'authors':item['author'],
                     'date':item['publishedAt']}
           all_data.append(news_info) 
           
        return all_data  
    else:
        print("Error fetching news:", data)
        return []

def save_news_to_csv(articles, filename):
    df2 = pd.DataFrame(articles)
    df2['date']=pd.to_datetime(df2['date'], utc=True)
    df2['date'] = df2['date'].dt.strftime('%d-%m-%Y')
    df2=df2.dropna(how='any',subset=['headline','category','date'])
    df2.to_csv(r'c:\users\91904\news-category1.csv')
    
    
    
    
    print(f'News data saved to {filename}')
cat=['SPORTS','EDUCATION','BUSINESS','POLITICS','TECH','SCIENCE','MONEY'] 

df1=[]

for i in cat:
    dff=[]
    dff=get_news(i,'7.2.2025','7.6.2025','YOUR API KEY')  
    
    
   
    if dff is not None: 
       
        df1.extend(dff) 
       
save_news_to_csv(df1,'news-category1.csv')

News data saved to news-category1.csv


In [43]:
df3=pd.read_csv(r'c:\users\91904\after-news.csv')
df4=pd.read_csv(r'c:\users\91904\news-category1.csv')
df3.drop('Unnamed: 0',axis=1,inplace=True)
df3['date']=pd.to_datetime(df3['date'])
df4.drop('Unnamed: 0',axis=1,inplace=True)
df4['date']=pd.to_datetime(df4['date'])
combined_df = pd.concat([df3, df4], ignore_index=True)


# Save to a new CSV
combined_df.to_csv('combined.csv', index=False)
print('combined csv file created successfully')

combined csv file created successfully


In [44]:
file = r"C:\Users\91904\combined.csv"
s3.upload_file(file, 'news-senticonomy-bucket', 'combined.csv')


In [47]:
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine
host="YOUR AMAZON HOST"
port=3306
username="admin"
pa="Jeyavani"
db="news"
my_conn = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(username,pa,host,port,db))

In [ ]:

df=pd.read_csv(r'C:\Users\91904\combined.csv')


df.to_sql("news_sentiment",my_conn,if_exists="replace",index=False)
